In [1]:
from util import load_user_data
import pandas as pd
import numpy as np

# Load explanation understanding, no control condition
valid_users, tp_data, _ = load_user_data(filename="../data/xailabdata_all.csv")
valid_users_2, tp_data_2, _ = load_user_data(filename="../data/xailabdata_llm_agent.csv")

valid_users = valid_users | valid_users_2
tp_data.update(tp_data_2)
variable_dict = {}
variable_dict["condition"] = []
understanding_variable_dict = {}
understanding_variable_dict["condition"] = []
explanation_understanding_dimensions = ["perceived_feature_understanding", "completeness", "coherence", "clarity", \
    "learning_effect", "understanding_of_system", "usefulness_explanation", "objective_feature_understanding"]
covariates_dimensions = ["Propensity to Trust", "Familiarity", "ATI", "mlbackground"]
trust_dimensions = ["Reliability/Competence", "Understandability/Predictability", "Trust in Automation"]
reliance_dimensions = ["accuracy", "accuracy-wid", "agreement_fraction", "switch_fraction", "RAIR", "RSR"]
for dimension in reliance_dimensions:
    variable_dict[dimension] = []
for dimension in trust_dimensions:
    variable_dict[dimension] = []
for dimension in covariates_dimensions:
    variable_dict[dimension] = []
    understanding_variable_dict[dimension] = []
for dimension in explanation_understanding_dimensions:
    understanding_variable_dict[dimension] = []
for user in valid_users:
    tp_condition = tp_data[user]["condition"]
    if tp_condition == "control":
        continue
    variable_dict["condition"].append(tp_condition)

    # for users in all conditions, load reliance
    reliance_measures = tp_data[user]["reliance_measures"]
    for dimension in reliance_dimensions:
        variable_dict[dimension].append(reliance_measures[dimension])
    # for participants in all conditions, load covariates and trust
    trust = tp_data[user]["Trust_in_automation"]
    for dimension in trust_dimensions:
        variable_dict[dimension].append(trust[dimension])
    for dimension in ["Propensity to Trust", "Familiarity"]:
        variable_dict[dimension].append(trust[dimension])
    for dimension in ["ATI", "mlbackground"]:
        variable_dict[dimension].append(tp_data[user][dimension])
    if tp_condition == "control":
        continue
    # for participants in dashboard, chatxai and chatxaiboost conditions, load explanation udnerstanding
    explanation_understanding = tp_data[user]["explanation_understanding"]
    understanding_variable_dict["condition"].append(tp_condition)
    for dimension in explanation_understanding_dimensions:
        understanding_variable_dict[dimension].append(explanation_understanding[dimension])
    for dimension in ["Propensity to Trust", "Familiarity"]:
        understanding_variable_dict[dimension].append(trust[dimension])
    for dimension in ["ATI", "mlbackground"]:
        understanding_variable_dict[dimension].append(tp_data[user][dimension])

# for dimension in variable_dict:
#     print(dimension, len(variable_dict[dimension]))

df = pd.DataFrame(variable_dict)
df2 = pd.DataFrame(understanding_variable_dict)
print(df.shape)

245 valid participants
{'control': 61, 'dashboard': 61, 'chatxai': 62, 'chatxaiboost': 61, 'chatxaiAuto': 0}
91 participants blindly rely on AI advice
{'control': 8, 'dashboard': 27, 'chatxai': 32, 'chatxaiboost': 24, 'chatxaiAuto': 0}
61 valid participants
{'control': 0, 'dashboard': 0, 'chatxai': 0, 'chatxaiboost': 0, 'chatxaiAuto': 61}
25 participants blindly rely on AI advice
{'control': 0, 'dashboard': 0, 'chatxai': 0, 'chatxaiboost': 0, 'chatxaiAuto': 25}
(245, 14)


In [2]:
from scipy.stats import spearmanr

print("Order for reliance measures", reliance_dimensions)
str_dict = {}
for dv in explanation_understanding_dimensions:
    str_dict[dv] = "{} ".format(dv)
    print(dv)
    for reliance_measure in reliance_dimensions:
        # print(len(understanding_variable_dict[dv]), understanding_variable_dict[dv])
        # print(len(variable_dict[reliance_measure]), variable_dict[reliance_measure])
        correlation, pvalue = spearmanr(understanding_variable_dict[dv], variable_dict[reliance_measure])
        # if pvalue < 0.05 / 4:
        #     print("Variable {} and variable {} have spearman correlation {:.3f} and pvalue {:.3f}".format(dv, reliance_measure, correlation, pvalue))
        # str_dict[dv] += "& {} & {}".format("%.3f"%correlation, ("%.3f"%pvalue)[1:])
        if pvalue < 0.0125:
            str_dict[dv] += "& {} &".format("%.3f"%correlation)
            str_dict[dv] += "\\textbf{"
            str_dict[dv] += "{}".format(("%.3f"%pvalue)[1:])
            str_dict[dv] += "}$^{\dagger\dagger}$"
        elif pvalue < 0.05:
            str_dict[dv] += "& {} & {}".format("%.3f"%correlation, ("%.3f"%pvalue)[1:])
            str_dict[dv] += "{\dagger}$"
        else:
            str_dict[dv] += "& {} & {}".format("%.3f"%correlation, ("%.3f"%pvalue)[1:])
    str_dict[dv] += "\\\\"
    print(str_dict[dv])
print("-" * 17)

for dv in trust_dimensions:
    str_dict[dv] = "{} ".format(dv)
    print(dv)
    for reliance_measure in reliance_dimensions:
        correlation, pvalue = spearmanr(variable_dict[dv], variable_dict[reliance_measure])
        # if pvalue < 0.05 / 4:
        #     print("Variable {} and variable {} have spearman correlation {:.3f} and pvalue {:.3f}".format(dv, reliance_measure, correlation, pvalue))
        # print(pvalue)
        if pvalue < 0.0125:
            str_dict[dv] += "& {} &".format("%.3f"%correlation)
            str_dict[dv] += "\\textbf{"
            str_dict[dv] += "{}".format(("%.3f"%pvalue)[1:])
            str_dict[dv] += "}$^{\dagger\dagger}$"
        elif pvalue < 0.05:
            str_dict[dv] += "& {} & {}".format("%.3f"%correlation, ("%.3f"%pvalue)[1:])
            str_dict[dv] += "{\dagger}$"
        else:
            str_dict[dv] += "& {} & {}".format("%.3f"%correlation, ("%.3f"%pvalue)[1:])
    str_dict[dv] += "\\\\"
    print(str_dict[dv])
print("-" * 17)

Order for reliance measures ['accuracy', 'accuracy-wid', 'agreement_fraction', 'switch_fraction', 'RAIR', 'RSR']
perceived_feature_understanding
perceived_feature_understanding & 0.045 & .484& -0.024 & .709& 0.254 &\textbf{.000}$^{\dagger\dagger}$& 0.117 & .067& 0.096 & .135& -0.293 &\textbf{.000}$^{\dagger\dagger}$\\
completeness
completeness & 0.050 & .435& 0.056 & .387& 0.146 & .022{\dagger}$& 0.142 & .026{\dagger}$& 0.157 & .014{\dagger}$& -0.170 &\textbf{.007}$^{\dagger\dagger}$\\
coherence
coherence & 0.107 & .095& -0.030 & .643& 0.270 &\textbf{.000}$^{\dagger\dagger}$& 0.068 & .286& 0.005 & .935& -0.218 &\textbf{.001}$^{\dagger\dagger}$\\
clarity
clarity & 0.002 & .973& -0.111 & .083& 0.190 &\textbf{.003}$^{\dagger\dagger}$& 0.081 & .204& 0.042 & .514& -0.235 &\textbf{.000}$^{\dagger\dagger}$\\
learning_effect
learning_effect & 0.084 & .192& -0.085 & .184& 0.170 &\textbf{.008}$^{\dagger\dagger}$& -0.006 & .931& 0.007 & .913& -0.135 & .035{\dagger}$\\
understanding_of_system
unde